In [1]:
# =============================================================================
# TASK 3: Sentiment Analysis & Correlation with Stock Returns
# KAIM Week 1 - Financial Sentiment Challenge
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("=" * 50)
print("TASK 3: SENTIMENT & CORRELATION ANALYSIS")
print("=" * 50)
print(f"\n✓ Libraries imported successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

TASK 3: SENTIMENT & CORRELATION ANALYSIS

✓ Libraries imported successfully!
Analysis Date: 2025-11-22 23:26
